# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

In [ ]:
! pip install Pillow numpy tk

In [ ]:
from tkinter import *
import random
from agents import *
import collections
collections.Callable = collections.abc.Callable





loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world



class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=',agent.location,' - Perceived StatusAtLoc=',  self.status)
        return (agent.location, self.status) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        
        location, status = percept
        print('Perceived Status: ', percept)

        if (status[location] == 'Dirty'): 
            return 'Suck'
        elif (location == loc_B): 
            if(status[loc_A]=='Dirty' and status[loc_C]=='Dirty'): 
                move=random.choice(['Right', 'Left']) 
                return move
            elif(status[loc_A]=='Dirty'): 
                return 'Left'
            else :
                return 'Right'
        elif(location == loc_A): 
            return 'Right' 
        elif(location == loc_C): 
            return 'Left'

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 100, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(110, 100, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 100, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(360, 100, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 1, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(250, 100, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    MyImage=PhotoImage(file='kirby4.png')
    
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


## Soluzione 1

nella sezione `agent`, con una nuova variabile **new move** gli viene assegnata una direzione casuale (**right** o **left**), che sarà il valore di return della funzione program dell'agente. Così quando di preme il tasto next, il valore di ritorno dell'agente verrà usato per il valore nella funzione `execute_action` 

sotto il codice modificato


In [5]:
from tkinter import *
import random
from agents import *
import collections
collections.Callable = collections.abc.Callable





loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world




class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=',agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        return (agent.location, self.status[agent.location]) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """agent(program)Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

#############################agent(program)######################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        
        location, status = percept
        print('Perceived Status: ', percept)
        new_move=random.choice(['Right','Left'])
        return new_move

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent ,MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(110, 110, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(360, 110, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(250, 110, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    MyImage=PhotoImage(file='kirby4.png')
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


## soluzione 2 
modifico in questo caso due funzioni: 
1) nella funzione `percept` invece di ritornare il luogo dove si trova l'agente e lo stato del luogo, ritorna comunque il luogo dove si trova l'aspirapolvere, ma ritorna invece lo stato di tutti e tre i luoghi. 
2) nella funzione program, ho agigunto nuove regole che determinano l'azione dell'agente in base allo stato dell'ambiente e alla sua posizione: 
    - se si strova in una casella sporca, ovviamente **aspira** 
    - se si trova in una casella pulita, si comporta in base a quale casella si trova  
        - caso 1: casella centrale. 
            in questo caso se una delle celle adiacenti è sporca, l'agente si sposta nella casella da pulire, 
            se invece sono tuti e due sporche, allora sceglie a caso in quale casella andare 
        - caso 2: caselle laterali. 
            in questo caso invece, l'agente si sposta al centro indipendentemente dallo stato della casella centrale.

di sotto il codice modificato

In [ ]:
from tkinter import *
import random
from agents import *
import collections
collections.Callable = collections.abc.Callable





loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world



class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=',agent.location,' - Perceived StatusAtLoc=',  self.status)
        return (agent.location, self.status) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        
        location, status = percept
        print('Perceived Status: ', percept)

        if (status[location] == 'Dirty'): 
            return 'Suck'
        elif (location == loc_B): 
            if(status[loc_A]=='Dirty' and status[loc_C]=='Dirty'): 
                move=random.choice(['Right', 'Left']) 
                return move
            elif(status[loc_A]=='Dirty'): 
                return 'Left'
            else :
                return 'Right'
        elif(location == loc_A): 
            return 'Right' 
        elif(location == loc_C): 
            return 'Left'

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(110, 110, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(360, 110, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 110, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(250, 110, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    MyImage=PhotoImage(file='kirby4.png')
    
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


## soluzione 3 e 4
ho modificato l'interfaccia graica in modo da transformare l'ambiente in una matrice 2x2, aggiungendo una nuova variabile `loc_D` come 'quarto' tassello della matrice (gli altri tre ho riusato le variabili `loc_A`, `loc_B` etc.), di conseguenza ho aggiunto un nuovo bottone nella gui, e ho risistemato le posizioni dei bottoni in modo da farlo sembrare una matrice. Oltre a ciò ho modificato i movimenti possibili dell'agente aggoiungendo le azioni `up` e `down`, e modificato le già presenti azioni `Right` e `left`. 
Fatto ciò ho modificato la funzione `program` dell'agente in modo che avesse lo stesso comportamento nell'esercizio 2. 
 
per l'esercizio 4 ho aggiunto una variabile, dove ad ogni azione eseguita prima di eseguirla genera un numero intero di valore casuale tra 1 e 10, e se assume il valore 1, non esegue l'azione e stampa 'action executed: failed', altrimenti esegue l'azione desiderata


di sotto il codice modificato 


In [ ]:
from tkinter import *
import random
from agents import *
import collections
collections.Callable = collections.abc.Callable





loc_A, loc_B, loc_C, loc_D = (0, 0), (0, 1), (1,0), (1,1)   # The three locations for the Vacuum world



class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean',
                       loc_D: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=',agent.location,' - Perceived StatusAtLoc=',  self.status)
        return (agent.location, self.status) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        n = random.randint(1,10) 
        if (n==1):
            print('Executing action: failed')
        else: 
            print('Executing action: ', action)
            if action == 'Right':
                if agent.location == loc_A:
                    agent.location=loc_B
                elif agent.location == loc_C:
                    agent.location=loc_D
                agent.performance -= 1
            elif action == 'Left':
                if agent.location == loc_B:
                    agent.location = loc_A
                elif agent.location == loc_D:
                    agent.location=loc_C
                agent.performance -= 1
            elif action == 'Up': 
                if agent.location == loc_C: 
                    agent.location = loc_A 
                elif agent.location == loc_D: 
                    agent.location = loc_B
                agent.performance -= 1
            elif action == 'Down': 
                if agent.location == loc_A: 
                    agent.location = loc_C 
                elif agent.location == loc_B: 
                    agent.location = loc_D
                agent.performance -= 1
            elif action == 'Suck':
                if self.status[agent.location] == 'Dirty':
                    if agent.location == loc_A:
                        self.buttons[0].config(bg='white', activebackground='light grey')
                    elif agent.location == loc_B:
                        self.buttons[1].config(bg='white', activebackground='light grey')
                    elif agent.location == loc_C:
                        self.buttons[2].config(bg='white', activebackground='light grey')
                    else: 
                        self.buttons[3].config(bg='white', activebackground='light grey')
                    agent.performance += 10
                self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A,loc_B,loc_C,loc_D])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_upper_left = Button(self.root, height=3, width=9, padx=2, pady=2, bg='white')
        button_upper_left.config(command=lambda btn=button_upper_left: self.dirt_switch(btn))
        self.buttons.append(button_upper_left)
        button_upper_left_window = self.canvas.create_window(130, 110, anchor=N, window=button_upper_left)

        button_upper_right = Button(self.root, height=3, width=9, padx=2, pady=2, bg='white')
        button_upper_right.config(command=lambda btn=button_upper_right: self.dirt_switch(btn))
        self.buttons.append(button_upper_right)
        button_upper_right_window = self.canvas.create_window(370, 110, anchor=N, window=button_upper_right)

        button_lower_left= Button(self.root, height=3, width=9, padx=2, pady=2, bg='white')
        button_lower_left.config(command=lambda btn=button_lower_left: self.dirt_switch(btn))
        self.buttons.append(button_lower_left)
        button_lower_left_window = self.canvas.create_window(130, 300, anchor=N, window=button_lower_left)
        
        button_lower_right = Button(self.root, height=3, width=9, padx=2, pady=2, bg='white')
        button_lower_right.config(command=lambda btn=button_lower_right: self.dirt_switch(btn))
        self.buttons.append(button_lower_right)
        button_lower_right_window = self.canvas.create_window(370, 300, anchor=N, window=button_lower_right)


    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            elif i == 2:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'
            else: 
                if btn['bg'] == 'white':
                    self.status[loc_D] = 'Clean'
                else:
                    self.status[loc_D] = 'Dirty'


    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        
        location, status = percept
        print('Perceived Status: ', percept)

        if (status[location] == 'Dirty'): 
            return 'Suck'
        elif (location == loc_A):
            if (status[loc_B] == 'Dirty'): 
                return 'Right'
            elif (status[loc_C] == 'Dirty'): 
                return 'Down'
            else: 
                return random.choice(['Right', 'Down'])
        elif (location == loc_B):
            if (status[loc_A] == 'Dirty'): 
                return 'Left'
            elif (status[loc_D] == 'Dirty'): 
                return 'Down'
            else: 
                return random.choice(['Left', 'Down'])
        elif (location == loc_C):
            if (status[loc_A] == 'Dirty'): 
                return 'Up'
            elif (status[loc_D] == 'Dirty'): 
                return 'Right'
            else: 
                return random.choice(['Right', 'Up']) 
        elif (location == loc_D): 
            if (status[loc_B] == 'Dirty'): 
                return 'Up'
            elif (status[loc_C] == 'Dirty'): 
                return 'Left'
            else: 
                return random.choice(['Left', 'Up'])
        
    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(130, 60, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(130, 60, image=MyImage)
        
    elif agent.location == (0, 1):
        env.text = env.canvas.create_text(370, 60, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(370, 60, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(130, 250, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(130, 250, image=MyImage)
    elif agent.location == (1, 1):
        env.text = env.canvas.create_text(370, 250, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(370, 250, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (0,1) and before_step == (0,0):
            env.canvas.move(env.text, 240, 0)
            env.canvas.move(env.agent_logo, 240, 0)
        elif agent.location == (0,0) and  before_step == (0,1): 
            env.canvas.move(env.text, -240, 0)
            env.canvas.move(env.agent_logo, -240, 0)
        elif agent.location == (1, 1) and  before_step == (1,0): 
            env.canvas.move(env.text, 240, 0)
            env.canvas.move(env.agent_logo, 240, 0)
        elif agent.location == (1, 0) and before_step == (1,1):
            env.canvas.move(env.text, -240, 0)
            env.canvas.move(env.agent_logo, -240, 0)
        elif agent.location == (1, 0) and before_step == (0,0):
            env.canvas.move(env.text, 0, 190)
            env.canvas.move(env.agent_logo, 0, 190)
        elif agent.location == (0,0) and before_step == (1,0):
            env.canvas.move(env.text, 0, -190)
            env.canvas.move(env.agent_logo, 0, -190)
        elif agent.location == (1, 1) and before_step == (0,1):
            env.canvas.move(env.text, 0, 190)
            env.canvas.move(env.agent_logo, 0, 190)
        elif agent.location == (0,1) and before_step == (1,1):
            env.canvas.move(env.text, 0, -190)
            env.canvas.move(env.agent_logo, 0, -190)
        

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    MyImage=PhotoImage(file='kirby4.png')
    
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


Perceived Location= (1, 0)  - Perceived StatusAtLoc= {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'}
Perceived Status:  ((1, 0), {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'})
Executing action: failed
Current Environment Status:  {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'}
Current Agent Location:  (1, 0)
Perceived Location= (1, 0)  - Perceived StatusAtLoc= {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'}
Perceived Status:  ((1, 0), {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'})
Executing action: failed
Current Environment Status:  {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'}
Current Agent Location:  (1, 0)
Perceived Location= (1, 0)  - Perceived StatusAtLoc= {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'}
Perceived Status:  ((1, 0), {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Clean'})
Executing action: failed
Curre